In [1]:
import tensorly as tl
import torch
import torch.nn.functional as F
from torch import nn
import numpy as np
import statistics
import torch.optim

In [2]:
class Lambda(nn.Module):
    def __init__(self):
        super().__init__()
        self.func = func
    
    def forward(self, x):
        return self.func(x)

In [3]:
tl.set_backend('numpy')
tensor_data = torch.FloatTensor(np.random.rand(1,1,30,500))
split = tensor_data.shape[3]//4

In [4]:
train_tensor = tensor_data[:,:,:,0:split*2]
val_tensor = tensor_data[:,:,:,split*2:split*3]
test_tensor = tensor_data[:,:,:,split*3:tensor_data.shape[3]]

In [5]:
class CNN_Extractor(nn.Module):
    def __init__(self, w, R, gKernel, lKernel):
        super().__init__()
        #self.local_kernel = local_kernel
        #self.global_kernel =  global_kernel
        self.w = w
        self.R = R
        self.gKernel = gKernel
        self.lKernel = lKernel
        self.conv2d_local = nn.Conv2d(1,1,(self.R,self.lKernel)) # 30x1
        self.conv2d_global = nn.Conv2d(1,1,(self.R,self.gKernel)) # 30x5
        self.fc = nn.Linear(in_features=2*self.w-lKernel-gKernel+2, out_features=self.R) 
            # Change this number of out_features to forecast longer time
            
    def forward(self, x):
        _local = self.conv2d_local(x)
        _global = self.conv2d_global(x)
        x = torch.cat((_local,_global),3)
        x = self.fc(x)
        x = torch.tanh(x)
        return x

In [6]:
# Hyperparameters
learning_rate = 1e-3
train_len = train_tensor.shape[3]
val_len = val_tensor.shape[3]
test_len = test_tensor.shape[3]
epochs = 100
# Create Model
cnn_extractor = CNN_Extractor(10,30,5,1)
loss_fnc = nn.MSELoss()
optimizer = torch.optim.SGD(cnn_extractor.parameters(), lr=learning_rate, momentum=0.9)


In [7]:
# Train and evaluation
for epoch in range(epochs):
    losses = torch.tensor([0])
    for i in range(train_len-10):
        train_input = train_tensor[:,:,:,i:i+10] #input Rx1
        train_target = torch.transpose(train_tensor[:,:,:,i+10:i+11],2,3)
        
        cnn_extractor.train()
        y_pred = cnn_extractor(train_input)
        loss = loss_fnc(train_target, y_pred)
        losses = torch.hstack((losses, loss))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch%2==0:
        cnn_extractor.eval()
        val_losses = torch.tensor([0])
        for i in range(val_len-10):
#             cnn_extractor.eval()
            val_input = val_tensor[:,:,:,i:i+10]
            val_target = torch.transpose(val_tensor[:,:,:,i+10:i+11],2,3)
            y_val = cnn_extractor(val_input)
            val_loss = loss_fnc(val_target, y_val)
            val_losses = torch.dstack((val_losses, val_loss))

        val_avg = torch.mean(val_losses)
        train_avg = torch.mean(losses)
        print(f'Epoch: {epoch}, Loss: {train_avg}\t\tEpoch: {epoch}, Val-loss: {val_avg}')



/home/seyunkim/anaconda3/envs/syenv/lib/python3.6/site-packages/torch/autograd/__init__.py:147: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


Epoch: 0, Loss: 0.3072214424610138		Epoch: 0, Val-loss: 0.1617903858423233
Epoch: 2, Loss: 0.09003757685422897		Epoch: 2, Val-loss: 0.08740904927253723
Epoch: 4, Loss: 0.08792639523744583		Epoch: 4, Val-loss: 0.08662678301334381
Epoch: 6, Loss: 0.08709518611431122		Epoch: 6, Val-loss: 0.0859762653708458
Epoch: 8, Loss: 0.08641600608825684		Epoch: 8, Val-loss: 0.08543183654546738
Epoch: 10, Loss: 0.08585486561059952		Epoch: 10, Val-loss: 0.08498255908489227
Epoch: 12, Loss: 0.0853872150182724		Epoch: 12, Val-loss: 0.084610715508461
Epoch: 14, Loss: 0.08499405533075333		Epoch: 14, Val-loss: 0.08430179953575134
Epoch: 16, Loss: 0.08466073870658875		Epoch: 16, Val-loss: 0.08404428511857986
Epoch: 18, Loss: 0.08437579870223999		Epoch: 18, Val-loss: 0.08382895588874817
Epoch: 20, Loss: 0.08413026481866837		Epoch: 20, Val-loss: 0.08364846557378769
Epoch: 22, Loss: 0.08391702175140381		Epoch: 22, Val-loss: 0.08349687606096268
Epoch: 24, Loss: 0.08373041450977325		Epoch: 24, Val-loss: 0.0833693

In [8]:
y_pred.shape

torch.Size([1, 1, 1, 30])

In [9]:
# test_data = test_tensor[:,:,:,0:5] 
# prediction = cnn_extractor(test_data)
# test_target = test_tensor[:,:,:,5:6]

In [10]:
def test_evaluate(data, comment):
    test_losses = torch.tensor([0])
    for i in range(data.shape[3]-10):
        test_data = data[:,:,:,i:i+10]
        test_target = torch.transpose(data[:,:,:,i+10:i+11],2,3)
        prediction = cnn_extractor(test_data)

        test_loss = loss_fnc(test_target, prediction)
        test_losses = torch.dstack((test_loss, test_losses))
    
    test_avg = torch.mean(test_losses)
    print(f'Comment: {comment}\nTest Loss : {test_avg}')

In [11]:
test_evaluate(test_tensor, "dawon2")

Comment: dawon2
Test Loss : 0.08292575180530548


In [12]:
test_evaluate(test_tensor, "dawon")

Comment: dawon
Test Loss : 0.08292575180530548


In [13]:
test_evaluate(test_tensor, "Global+local")

Comment: Global+local
Test Loss : 0.08292575180530548


In [14]:
test_evaluate(test_tensor, "Larger dataset")

Comment: Larger dataset
Test Loss : 0.08292575180530548


In [15]:
test_evaluate(test_tensor, "Without ReLu")

Comment: Without ReLu
Test Loss : 0.08292575180530548


In [16]:
# # Evaluate with test data
# test_losses = torch.tensor([0])
# for i in range(test_len-5):
#     test_data = test_tensor[:,:,:,i:i+5]
#     test_target = torch.transpose(test_tensor[:,:,:,i+5:i+6],2,3)
#     prediction = cnn_extractor(test_data)
    
#     test_loss = loss_fnc(test_target, prediction)
#     test_losses = torch.dstack((test_loss, test_losses))

# test_avg = torch.mean(test_losses)
# print(f'Test Loss : {test_avg}')
    